In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn import tree

In [2]:
beanData = pd.read_csv('Dry_Bean_Dataset.xlsx')


X = beanData.iloc[:,0:15].values
y = beanData.iloc[:,16].values
beanData.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [3]:
from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
y = encodeLabel.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size= 0.63, random_state=1738)


# SUPPORT VECTOR MACHINE TRIAL ONE ON BEANS DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = beanData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:15].values
    ySet = random5000DataPoints.iloc[:,16].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.4min finished


In [6]:
TRIAL_1_SVM.cv_results_

{'mean_fit_time': array([4.93394299, 4.87859545, 4.96937356, 4.98258486, 4.87599297,
        5.10208802, 5.02251902, 5.05794978, 5.0739634 , 4.97537885,
        4.92833805, 4.97537847, 4.93874717, 4.98038301, 4.89881325,
        4.70054259, 3.36499391, 5.05774965, 4.85127177, 5.13421793,
        4.97778087, 5.14602594, 5.24140735, 4.9650702 , 2.30948634,
        1.4693634 , 3.42322102, 5.55267482, 5.37372117, 5.19756956,
        5.12831025, 4.57383313, 2.03765259, 0.91438632, 0.75554948,
        2.42338405, 5.10428939, 4.90241609, 5.13101249, 4.53349881,
        2.03024635, 0.96913395, 0.62583756, 0.83842158, 2.43889742,
        4.89190702, 4.85007091, 4.47354727, 1.9352643 , 0.90137506,
        0.68138595, 0.8157011 , 1.16450138, 2.6110455 , 4.79422312,
        4.43751645, 1.96498995, 0.94110961, 0.69209514, 0.85523529,
        1.31062622, 1.77362556, 2.74215827, 4.43941755, 2.06497622,
        0.92849855, 0.68188648, 0.87665334, 1.3032207 , 2.66479206,
        2.34081292, 2.77488651,

In [7]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [8]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_f1_micro']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'poly'}

In [9]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [10]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best NEG LOG LOSS hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best F1 hyperparameters :0.9135860058309038
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best ROC_AUC hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL TWO ON BEANS DATASET

In [11]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = beanData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:15].values
    ySet = random5000DataPoints.iloc[:,16].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', MinMaxScaler(feature_range = (0, 1))),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.5min finished


In [12]:
TRIAL_2_SVM.cv_results_

{'mean_fit_time': array([3.85881824, 4.81003637, 4.71745696, 4.65380206, 4.61546917,
        4.76599827, 4.64419403, 4.60105662, 4.67422013, 3.87142906,
        4.71665592, 4.64969883, 4.65550375, 4.72196155, 4.8851018 ,
        5.01444664, 4.81694288, 4.24414992, 4.38336949, 5.12230501,
        4.97637901, 5.27803884, 4.83325634, 4.62777977, 4.75779204,
        3.43105054, 1.42002115, 3.86422305, 4.68472881, 4.6669136 ,
        4.61526899, 4.60566072, 4.58804579, 3.21476417, 1.32073579,
        0.7051065 , 3.99503598, 4.95806403, 5.20107288, 4.77350488,
        4.80142903, 3.28152189, 1.35256352, 0.7164165 , 0.58810573,
        3.90655951, 4.82044539, 4.75789161, 4.74398012, 3.40943208,
        1.41541748, 0.71631598, 0.60782261, 0.90988259, 4.10513058,
        4.95385938, 5.02642241, 3.31565161, 1.30792456, 0.67698183,
        0.62353621, 0.88386025, 1.47326736, 4.27697906, 5.14372344,
        3.64153233, 1.59677291, 0.70730834, 0.65766521, 0.94501305,
        1.72998796, 3.06433511,

In [13]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [14]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_f1_micro']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [15]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [16]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best NEG LOG LOSS hyperparameters :0.9259475218658892
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best F1 hyperparameters :0.9219825072886297
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best ROC_AUC hyperparameters :0.9259475218658892
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL THREE ON BEANS DATASET

In [17]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = beanData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:15].values
    ySet = random5000DataPoints.iloc[:,16].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   57.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.2min finished


In [18]:
TRIAL_3_SVM.cv_results_

{'mean_fit_time': array([4.79482231, 4.70844865, 4.69053378, 4.69663887, 4.67912393,
        4.67482033, 4.80242972, 4.86718535, 4.76199527, 4.78401432,
        4.74037657, 4.76539845, 4.71145186, 4.84006248, 5.07566481,
        4.91312547, 3.31705298, 5.07536449, 5.0548471 , 4.9380466 ,
        4.99939909, 4.92463541, 4.99649706, 4.35474491, 2.01303115,
        1.22505364, 3.19244504, 4.79922676, 4.69053383, 4.69033341,
        4.77340508, 4.43241196, 2.00542502, 1.07602553, 0.8168026 ,
        2.54328709, 4.8402626 , 4.78481674, 4.74518099, 4.22112989,
        1.87421169, 0.85573597, 0.5942111 , 0.79618483, 2.48103385,
        4.75468888, 4.69884109, 4.32311788, 1.9467742 , 0.97163486,
        0.64095144, 0.78117232, 1.18561959, 2.6239563 , 4.7436799 ,
        4.3442358 , 1.87120919, 0.88686242, 0.63975043, 0.84472666,
        1.24136729, 1.76395831, 2.82172685, 4.33392735, 2.03805265,
        1.01186962, 0.67468014, 0.86864724, 1.44574366, 2.65708485,
        2.2715529 , 2.68480878,

In [19]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [20]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_f1_micro']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1000,
 'classifier__gamma': 0.01,
 'classifier__kernel': 'rbf'}

In [21]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [23]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best NEG LOG LOSS hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best F1 hyperparameters :0.9250145772594752
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best ROC_AUC hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL FOUR ON BEANS DATASET

In [24]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = beanData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:15].values
    ySet = random5000DataPoints.iloc[:,16].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.2min finished


In [25]:
TRIAL_4_SVM.cv_results_

{'mean_fit_time': array([4.82575026, 4.65990663, 4.74528003, 4.63198304, 4.64059076,
        4.70404568, 4.74167767, 5.078968  , 5.03813257, 4.94865561,
        4.70204353, 4.74327908, 4.77400551, 4.72476344, 4.82595057,
        4.68112535, 3.33096442, 4.83625903, 4.94715486, 4.86748633,
        4.93454328, 5.07346249, 4.92693653, 4.35724721, 2.02874494,
        1.23796439, 3.23648319, 4.89811225, 4.97157578, 5.25692081,
        5.32477908, 4.87459197, 2.05326562, 1.01547327, 0.81590133,
        2.84674773, 5.226895  , 5.07336326, 5.33058419, 4.50147104,
        2.12482729, 0.91108336, 0.56568651, 0.75154629, 2.59122825,
        4.96466961, 4.97187572, 4.46073627, 1.83878102, 0.84642811,
        0.58640418, 0.69709883, 1.0720221 , 2.67970471, 4.93994827,
        4.39407878, 1.9386673 , 0.87425179, 0.60101652, 0.77526646,
        1.14498448, 1.73359094, 2.85075173, 4.4223033 , 2.04135551,
        0.952319  , 0.64715667, 0.83581858, 1.2412674 , 2.4016654 ,
        2.29417286, 2.78639641,

In [26]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [27]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_f1_micro']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1000,
 'classifier__gamma': 0.01,
 'classifier__kernel': 'rbf'}

In [28]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [29]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best NEG LOG LOSS hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best F1 hyperparameters :0.9250145772594752
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best ROC_AUC hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL FIVE ON BEANS DATASET

In [30]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = beanData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:15].values
    ySet = random5000DataPoints.iloc[:,16].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   49.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.2min finished


In [31]:
TRIAL_5_SVM.cv_results_

{'mean_fit_time': array([4.69864092, 4.55451684, 4.5773365 , 4.69263492, 4.73797417,
        4.67772303, 4.76219544, 4.92843857, 4.90621958, 4.80523248,
        4.91732912, 4.87579303, 4.97347693, 4.71665611, 4.69243531,
        4.47715011, 3.30163932, 4.75649042, 4.7714036 , 4.6857296 ,
        4.65089965, 4.67241802, 4.73136859, 4.28198233, 1.94016867,
        1.1855195 , 3.1051703 , 4.70244393, 4.62287545, 4.64479475,
        4.8452672 , 4.62958131, 2.04575934, 1.02418094, 0.76776018,
        2.4795321 , 4.88099737, 4.91903048, 4.87919588, 4.38947477,
        1.89412889, 0.83501787, 0.58260078, 0.75384812, 2.57991843,
        4.91182404, 4.93314233, 4.56052198, 1.90523815, 0.87004843,
        0.59771395, 0.69109383, 1.07522464, 2.53868313, 4.75849214,
        4.17999439, 1.82897329, 0.86344247, 0.59971571, 0.74033613,
        1.12646847, 1.68284702, 2.66649318, 4.31280885, 1.81796355,
        0.8806571 , 0.62954178, 0.81149759, 1.24747276, 2.43199134,
        2.1867806 , 2.65508327,

In [32]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [33]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_f1_micro']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 0.0001,
 'classifier__kernel': 'rbf'}

In [34]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [35]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best NEG LOG LOSS hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best F1 hyperparameters :0.9239650145772594
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best ROC_AUC hyperparameters :0.926530612244898
------------------------------------------------------------------------------------------------
